# Course report for DD3436 (implementing `sillymap`)

This course consisted of hands-on exercises, implementing and improving a burrows-wheeler short read mapper. To simplify the task, the mapper created here were only able to find the position for exactly matching reads against a single reference sequence, hence the name `sillymap` were chosen.

The four exercises were as follows:

### Excercise 1:
A repository on Github with a Readme, license and setup.py. The program itself only prints a friendly message. Bonus: Travis CI.

### Exercise 2:
A release on github where the program can map a few exactly matching short reads against a longer sequence, using the burrows wheeler transform and the FM-index (http://alexbowe.com/fm-index/) where rank can be stored as the table Occ in (https://en.wikipedia.org/wiki/FM-index). This assignment should include a test suite.

### Exercise 3:
Using the test suite for assignment 2, modify the code to make it run faster/more efficiently. Create two versions:
 1. Make it run with pypy.
 2. Using numpy, scipy or other libraries to speed up the code.

Both scripts should pass integration tests in Travis.

Make a report explaining the optimizations and the improvements.

### Exercise 4:
Parallelize the code using both:
 1. Multiple cores on one computer.
 2. Using multiple computers (MPI).

Make a report explaining the optimizations and the improvements.

## Exercise 1
The repository was created and released as assignment 1 on [github](https://github.com/alneberg/sillymap/releases).

The basic structure of the repository was constructed for this exercise with one subdirectory for the main source code (which was only an `__init__.py` file at this point and one subdirectory containing only the executable. The executable which can be seen below printed the intended outcome for the two main methods of `sillymap` but did not do anything.

To complete the package and to make it installable, a `setup.py` file was created. This enabled the package to be installed by running `python setup.py install` and that would also make the executable `sillymap` available to the user. 

## Exercise 2
For this exercise, a first working version of the program was constructed and completed with a test suite. I created the test suite after the implementation as I've found it much faster to implement afterwards. It's also, in my opinion, more useful in the case of a refactoring rather than during the first implementation. The first implementation was written without much thought on efficiency. For example the burrows wheeler transform was calculated accordingly:

I think it was a good idea to do it this way first, to keep the code as simple as possible when the main focus was to make it work at all.

The test cases were split up into unit tests and integration tests. The unit tests checked edge cases and normal cases for individual functions such as the `burrows_wheeler` transform shown above. The integration tests simulated a run with a reference file and a set of very few reads. These tests were automated to be run on Travis using the settings file:

## Exercise 3
I created several reference sequences of different sizes and generated perfect reads from these to use as test data. When trying to index a 1M long sequence with the first version of the code, my laptop crashed and indeed, the memory usage was higher than 128G. Using [memory profiler](https://pypi.python.org/pypi/memory_profiler) I identified one bottleneck in the `burrows_wheeler` method shown above. The list `all_permutations` will store a full length sequence for every position within the input sequence, which is obviously quadratic in memory. On top of this, the default sort method seemed slightly waistful of memory.

To avoid creating all of the suffices, they were instead compared using the first 10 bases in most cases during the sorting, except when the first ten bases were identical between two suffices, when the complete suffices were extracted and compared. This is shown in the code below:

In [1]:
import matplotlib
%matplotlib inline